This is the file were using to test the llama api.

Create Api request object: 

We define the conent a

In [12]:
import pandas as pd
import numpy as np
import seaborn as sn
import json
import matplotlib.pyplot as plt
from collections import Counter
from llamaapi import LlamaAPI
from dotenv import load_dotenv
import os 
import requests

Creating the LLAMA object to start using the API

In [27]:
load_dotenv()
# Create a new LlamaAPI object
LLAMA_API_KEY = os.getenv("LLAMA_API_KEY")
LLAMA_API_URL = os.getenv("LLAMA_API_URL") 
llama = LlamaAPI(LLAMA_API_KEY)

We define the content as: What is the sector and industry of this job

We give the API the following infomration: job description, responsibilities, and skills 

In [20]:
api_request_json = {
  "model": "llama3.1-70b",
  "messages": [
    {"role": "user", "content": "What is the sector and industry for the following job: Skills: {skills}, Responsibilities: {responsibilities}, Description: {description}?"}, 
  ],
  "functions": [
    {
        "name": "get_sector_industry",
        "description": 
            "Get the sector and industry from this given job information such as skills, responsibilites, and description",
        "parameters": {
            "type": "object",
        "properties": { 
            "job_description": {
                "type": "string",
                "description": "The job description for the role"
            },
            "responsibilites": {
                "type": "string",
                "description": "The job responsibilites for the role"
            },
            "skills": {
                "type": "string",
                "description": "The required skills listed for the role"
            },
            "company": {
                "type": "string",
                "description": "The name of the company"
            },
        },
        "required": ["job description", "responsibilites", "skills"] 
        }
    }
  ],
}


Prepare json object for the api request

In [34]:
df = pd.read_csv("job_descriptions.csv")

selected_columns = df[['skills', 'Responsibilities', 'Job Description', 'Company']]
df = pd.DataFrame(selected_columns)
df = df.head(50)
print(df)

                                               skills  \
0   Social media platforms (e.g., Facebook, Twitte...   
1   HTML, CSS, JavaScript Frontend frameworks (e.g...   
2   Quality control processes and methodologies St...   
3   Wireless network design and architecture Wi-Fi...   
4   Event planning Conference logistics Budget man...   
5   Quality assurance processes Testing methodolog...   
6   Teaching pedagogy Classroom management Curricu...   
7   UI design principles and best practices Graphi...   
8   Interaction design principles User behavior an...   
9   Wedding planning Vendor coordination Event man...   
10  Performance testing Load testing Stress testin...   
11  Family law Divorce proceedings Child custody M...   
12  Mechanical engineering CAD software (e.g., Sol...   
13  Network security protocols Intrusion detection...   
14  Account management Sales strategy Negotiation ...   
15  Product branding and marketing Product lifecyc...   
16  School social work principl

In [35]:
def prepare_json(df):
    # Convert DataFrame into JSON-ready format
    job_listings = []
    for index, row in df.iterrows():
        listing = {
            "skills": row['skills'],
            "responsibilities": row['Responsibilities'],
            "description": row['Job Description'],
            "company": row['Company']
        }
        job_listings.append(listing)
    
    # Return the JSON structure that will be sent to the API
    json_job = {
        "job_listings": job_listings  # List of jobs to be analyzed by Llama API
    }
    return json_job

# Prepare the JSON object
json_job = prepare_json(df)

# To see the structure of the JSON
print(json.dumps(json_job, indent=4))

{
    "job_listings": [
        {
            "skills": "Social media platforms (e.g., Facebook, Twitter, Instagram) Content creation and scheduling Social media analytics and insights Community engagement Paid social advertising",
            "responsibilities": "Manage and grow social media accounts, create engaging content, and interact with the online community. Develop social media content calendars and strategies. Monitor social media trends and engagement metrics.",
            "description": "Social Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.",
            "company": "Icahn Enterprises"
        },
        {
            "skills": "HTML, CSS, JavaScript Frontend frameworks (e.g., React, Angular) User experience (UX)",
            "responsibilities": "Design and code user interfaces for websites, ensuring a seamless and visually appe

In [37]:
print(f"API URL: {LLAMA_API_URL}")

def get_sector_industry(json_job):
    headers = {
        'Authorization': f'Bearer {LLAMA_API_KEY}', 
        'Content-Type': 'application/json'
    }
    
    try:
        # Send POST request to Llama API with the job data
        response = requests.post(LLAMA_API_URL, headers=headers, data=json.dumps(json_job))
        
        # Check if the response is successful
        if response.status_code == 200:
            # Parse the JSON response
            results = response.json()
            
            # Extract sector and industry information
            sectors = [res['sector'] for res in results['job_listings']]
            industries = [res['industry'] for res in results['job_listings']]
            
            return sectors, industries
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Using the function to get sector and industry for the job listings
sectors, industries = get_sector_industry(json_job)

# Adding the sector and industry back into the DataFrame
if sectors and industries:
    df['sector'] = sectors
    df['industry'] = industries

# Display the DataFrame with new columns
print(df)
# caroline says to not use a specific external url/database to query for info -- this is what the email and weather functions used but our case is different
# Need to create function to get skills, responsibilites, and description from job posting

API URL: https://llama.api/v1/predict
An error occurred: HTTPSConnectionPool(host='llama.api', port=443): Max retries exceeded with url: /v1/predict (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x137333220>: Failed to resolve 'llama.api' ([Errno 8] nodename nor servname provided, or not known)"))
                                               skills  \
0   Social media platforms (e.g., Facebook, Twitte...   
1   HTML, CSS, JavaScript Frontend frameworks (e.g...   
2   Quality control processes and methodologies St...   
3   Wireless network design and architecture Wi-Fi...   
4   Event planning Conference logistics Budget man...   
5   Quality assurance processes Testing methodolog...   
6   Teaching pedagogy Classroom management Curricu...   
7   UI design principles and best practices Graphi...   
8   Interaction design principles User behavior an...   
9   Wedding planning Vendor coordination Event man...   
10  Performance testing Load testing Stres